In [1]:
import sys
sys.path.insert(0, "../src/gen")
sys.path.insert(1, "../src/rte")
from pathlib import Path
from collections import Counter

import numpy as np
from scipy.special import softmax

from util import read_data, write_jsonl
from aggregate import generate_micro_macro_df, generate_doc_df

In [21]:
# constants
LOOKUP = {
    "verifiable": {"no": "NOT VERIFIABLE", "yes": "VERIFIABLE"},
    "label": {"nei": "NOT ENOUGH INFO", "r": "REFUTES", "s": "SUPPORTS"}
}

SEED = 123456789

LABEL2ID = {"SUPPORTS": 0, "NOT ENOUGH INFO": 1, "REFUTES": 2}
ID2LABEL = {0: "SUPPORTS", 1: "NOT ENOUGH INFO", 2: "REFUTES"}

# Init

In [3]:
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding,
    TextClassificationPipeline,
    pipeline
)

# Huggingface Init

## Model

In [4]:
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

In [5]:
model_checkpoint = "bert-base-uncased"
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint, 
        num_labels=3, 
        id2label=ID2LABEL, 
        label2id=LABEL2ID
    )

model = model_init()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, do_lower_case=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess(examples):
    return tokenizer(examples["evidence"], examples["claim"], max_length=512, truncation="only_first")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    results = {}
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average="macro"))
    
    return results

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
dataset = ["fever", "climatefever", "fever-climatefever"]
task = ["hp_tune", "output"]
doc_sent = ["doc", "sent"]

di = 0
ti = 0
ds = 0

model_store_path = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/models").joinpath(model_checkpoint)
model_store_path.mkdir(exist_ok=True)
model_store_path = model_store_path / f"{dataset[di]}-{model_checkpoint}-{doc_sent[ds]}-{task[ti]}"

## Dataset

In [7]:
datap = Path(f"/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/bert-data-{doc_sent[ds]}-evidence")

data = DatasetDict({
    "train": Dataset.from_list(read_data(datap / f"{dataset[di]}.train.n5.jsonl")),
    "validation": Dataset.from_list(read_data(datap / f"{dataset[di]}.dev.n5.jsonl")),
    "test": Dataset.from_list(read_data(datap / f"{dataset[di]}.test.n5.jsonl"))
}).map(preprocess, batched=True)

Map:   0%|          | 0/145449 [00:00<?, ? examples/s]

Map:   0%|          | 0/9999 [00:00<?, ? examples/s]

Map:   0%|          | 0/9999 [00:00<?, ? examples/s]

## Trainer

In [8]:
batch_size = 32
learning_rate = 2e-5
epoch = 4
metric_name = "f1"
warmup_ratio=0.1
save_steps=200
eval_steps=200

# Hyperparameter tuning

In [9]:
# shard the data if the dataset is large for hyperparameter tuning
shard = data["train"].num_rows > 50000
hp_tune_train = data["train"].shuffle(seed=SEED).shard(num_shards=5, index=1)

In [10]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=eval_steps,
    save_strategy = "no",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard",
    tf32=True
)

_ = model.train()
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=hp_tune_train if shard else data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [5e-5, 3e-5, 2e-5]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
    }

def compute_objective(metrics):
    return metrics["f1"]

In [12]:
best_run = trainer.hyperparameter_search(
    direction="maximize", 
    n_trials=10, 
    hp_space=optuna_hp_space
)

[I 2023-07-04 13:43:19,788] A new study created in memory with name: no-name-d00b05c7-1510-47de-b5f6-a7e264da984e
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.661620,0.658066,0.658066,0.659200,0.547713
400,No log,0.239154,0.912491,0.912491,0.916174,0.912087
600,0.438900,0.209630,0.925993,0.925993,0.928032,0.925833
800,0.438900,0.208217,0.931293,0.931293,0.933916,0.931267
1000,0.190900,0.211251,0.932693,0.932693,0.936083,0.932621
1200,0.190900,0.194829,0.939994,0.939994,0.941238,0.939965
1400,0.190900,0.242561,0.925193,0.925193,0.930704,0.924873
1600,0.134500,0.234694,0.933893,0.933893,0.937704,0.933789
1800,0.134500,0.201446,0.936894,0.936894,0.939571,0.936696
2000,0.107000,0.252698,0.936494,0.936494,0.939619,0.936383


[I 2023-07-04 13:50:43,919] Trial 0 finished with value: 3.7657467255346386 and parameters: {'learning_rate': 3e-05, 'num_train_epochs': 3}. Best is trial 0 with value: 3.7657467255346386.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (in

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.646420,0.655766,0.655766,0.489200,0.545109
400,No log,0.306013,0.878288,0.878288,0.891253,0.876036
600,0.500800,0.216311,0.921292,0.921292,0.922859,0.920955
800,0.500800,0.213145,0.925993,0.925993,0.929980,0.925847
1000,0.202700,0.203453,0.934693,0.934693,0.936586,0.934636
1200,0.202700,0.190880,0.939794,0.939794,0.940391,0.939739
1400,0.202700,0.211524,0.926593,0.926593,0.931494,0.926282
1600,0.147800,0.230081,0.930693,0.930693,0.934777,0.930575
1800,0.147800,0.220852,0.930793,0.930793,0.935333,0.930619
2000,0.118400,0.250252,0.933593,0.933593,0.936882,0.933674


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-04 14:00:37,725] Trial 1 finished with value: 3.7550912675037296 and parameters: {'learning_rate': 2e-05, 'num_train_epochs': 4}. Best is trial 0 with value: 3.7657467255346386.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassi

...

[I 2023-07-04 14:33:29,292] Trial 7 pruned. 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.469518,0.732673,0.732673,0.831569,0.684342
400,No log,0.230421,0.921692,0.921692,0.922456,0.921592
600,0.400900,0.195518,0.927893,0.927893,0.930097,0.927742
800,0.400900,0.198819,0.935594,0.935594,0.938158,0.935490
1000,0.186600,0.208504,0.935594,0.935594,0.938557,0.935567
1200,0.186600,0.199093,0.940294,0.940294,0.941006,0.940311
1400,0.186600,0.198199,0.940694,0.940694,0.942118,0.940634
1600,0.126000,0.234338,0.934093,0.934093,0.937923,0.933980
1800,0.126000,0.207779,0.939494,0.939494,0.942063,0.939446


[I 2023-07-04 14:38:27,071] Trial 8 finished with value: 3.7604967047778217 and parameters: {'learning_rate': 3e-05, 'num_train_epochs': 2}. Best is trial 0 with value: 3.7657467255346386.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (in

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.469518,0.732673,0.732673,0.831569,0.684342
400,No log,0.230421,0.921692,0.921692,0.922456,0.921592
600,0.400900,0.195518,0.927893,0.927893,0.930097,0.927742
800,0.400900,0.198819,0.935594,0.935594,0.938158,0.935490
1000,0.186600,0.208504,0.935594,0.935594,0.938557,0.935567
1200,0.186600,0.199093,0.940294,0.940294,0.941006,0.940311
1400,0.186600,0.198199,0.940694,0.940694,0.942118,0.940634
1600,0.126000,0.234338,0.934093,0.934093,0.937923,0.933980
1800,0.126000,0.207779,0.939494,0.939494,0.942063,0.939446


[I 2023-07-04 14:43:25,487] Trial 9 finished with value: 3.7604967047778217 and parameters: {'learning_rate': 3e-05, 'num_train_epochs': 2}. Best is trial 0 with value: 3.7657467255346386.


In [13]:
best_run

BestRun(run_id='0', objective=3.7657467255346386, hyperparameters={'learning_rate': 3e-05, 'num_train_epochs': 3}, run_summary=None)

## Train with best hyperparameters

In [14]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=eval_steps,
    save_strategy="steps",
    save_steps=save_steps,
    save_total_limit=5,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard",
    tf32=True
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)
    
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.724722,0.639464,0.639464,0.483425,0.532786
400,No log,0.594086,0.658666,0.658666,0.486251,0.546110
600,0.600800,0.378740,0.851985,0.851985,0.881005,0.847920
800,0.600800,0.220865,0.916392,0.916392,0.918620,0.915787
1000,0.243800,0.237452,0.915392,0.915392,0.922580,0.914607
1200,0.243800,0.215078,0.926193,0.926193,0.928079,0.926313
1400,0.243800,0.197244,0.926193,0.926193,0.929311,0.925924
1600,0.196800,0.227010,0.926693,0.926693,0.930919,0.926455
1800,0.196800,0.186896,0.935994,0.935994,0.936083,0.936036
2000,0.182700,0.185183,0.931593,0.931593,0.934586,0.931380


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=13638, training_loss=0.13386852515025263, metrics={'train_runtime': 2303.3965, 'train_samples_per_second': 189.436, 'train_steps_per_second': 5.921, 'total_flos': 5.8014335353860776e+16, 'train_loss': 0.13386852515025263, 'epoch': 3.0})

In [15]:
trainer.save_model(model_store_path.parent / (model_store_path.stem + ".out"))

# Evaluate

In [16]:
import pandas as pd
from sklearn.metrics import classification_report

## Test on validation data

In [17]:
preds = trainer.predict(data["validation"])
val = generate_doc_df(data["validation"], preds)

In [18]:
print(classification_report(y_true=val["actual"], y_pred=val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       1.00      1.00      1.00      3333
        REFUTES       0.96      0.90      0.93      3333
       SUPPORTS       0.91      0.96      0.93      3333

       accuracy                           0.95      9999
      macro avg       0.95      0.95      0.95      9999
   weighted avg       0.95      0.95      0.95      9999



## Test on test data

In [19]:
preds = trainer.predict(data["test"])
tes = generate_doc_df(data["test"], preds)

In [20]:
print(classification_report(y_true=tes["actual"], y_pred=tes["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       1.00      1.00      1.00      3333
        REFUTES       0.95      0.87      0.91      3333
       SUPPORTS       0.88      0.95      0.92      3333

       accuracy                           0.94      9999
      macro avg       0.94      0.94      0.94      9999
   weighted avg       0.94      0.94      0.94      9999

